In [24]:
import json
import pandas as pd

in_data = json.load(open('../Bob_translate_result/B_in_result.json'))
out_data = json.load(open('../Bob_translate_result/B_out_result.json'))
data = list(in_data.values()) + list(out_data.values())
label = [0 for _ in range(len(list(in_data.values())))] + [1 for _ in range(len(list(out_data.values())))]
data_dict = {'ori': [data[i][0] for i in range(len(data))], 'gt': [data[i][1] for i in range(len(data))], 'pred': [data[i][2] for i in range(len(data))], 'label': label}
df = pd.DataFrame(data_dict)
df.head()

,ori,gt,pred,label
0,86. 爲此，糧農組織將考慮就如何設計1996年及其後年份的活動所提出的下述建議：,"In this connection, FAO will consider the foll...","86. To this end, FAO will consider the followi...",0
1,(a) 應當進一步開發變化矩陣系列的統計設計和分析系統：,(a) Statistical designs and analytical systems...,(a) The statistical design and analysis system...,0
2,㈠ 利用輔助資料，例如，藉助美國諾阿的高級甚高分辨率輻射 計（一公里分辨率）以及各種數據庫（...,(i) To use the auxiliary information such as t...,"(i) To use complementary information, such as ...",0
3,㈡ 通過可減少森林面積變化數字差異的分層參數改進對變化的 估測，如人口統計學參數，經濟指示數...,(ii) To improve estimates for change by strati...,(ii) Improving the estimation of changes throu...,0
4,(b) 應當進一步推動採用可靠的監測程序，如相互依賴的遙感分析，以獲得可協助決策的針對具體地...,(b) Sound monitoring procedures such as interd...,"(b) The use of reliable monitoring procedures,...",0


In [25]:
from nltk.translate.bleu_score import modified_precision, sentence_bleu, SmoothingFunction

def modified_n_gram_precisions(ref, hypo, n):
    return float(modified_precision([ref.split()], hypo.split(), n))

def bleu_score(ref, hypo):
    chencherry = SmoothingFunction()
    return sentence_bleu([ref.split()], hypo.split(), smoothing_function=chencherry.method1)

df['1-gram'] = df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 1), axis=1)
df['2-gram'] = df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 2), axis=1)
df['3-gram'] = df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 3), axis=1)
df['4-gram'] = df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 4), axis=1)
df['bleu'] = df.apply(lambda r: bleu_score(r['gt'], r['pred']), axis=1)
df.head()

,ori,gt,pred,label,1-gram,2-gram,3-gram,4-gram,bleu
0,86. 爲此，糧農組織將考慮就如何設計1996年及其後年份的活動所提出的下述建議：,"In this connection, FAO will consider the foll...","86. To this end, FAO will consider the followi...",0,0.684211,0.555556,0.470588,0.375000,0.508918
1,(a) 應當進一步開發變化矩陣系列的統計設計和分析系統：,(a) Statistical designs and analytical systems...,(a) The statistical design and analysis system...,0,0.500000,0.200000,0.142857,0.076923,0.182071
2,㈠ 利用輔助資料，例如，藉助美國諾阿的高級甚高分辨率輻射 計（一公里分辨率）以及各種數據庫（...,(i) To use the auxiliary information such as t...,"(i) To use complementary information, such as ...",0,0.465116,0.190476,0.097561,0.025000,0.121242
3,㈡ 通過可減少森林面積變化數字差異的分層參數改進對變化的 估測，如人口統計學參數，經濟指示數...,(ii) To improve estimates for change by strati...,(ii) Improving the estimation of changes throu...,0,0.625000,0.260870,0.090909,0.047619,0.156343
4,(b) 應當進一步推動採用可靠的監測程序，如相互依賴的遙感分析，以獲得可協助決策的針對具體地...,(b) Sound monitoring procedures such as interd...,"(b) The use of reliable monitoring procedures,...",0,0.500000,0.200000,0.041667,0.000000,0.062779


In [26]:
from nltk.translate.bleu_score import corpus_bleu

def group_features(sample):
    l = pd.cut(sample['bleu'], 100).value_counts().sort_index(axis=0).tolist()
    a = sample['gt'].tolist()
    ref = [[i.split()] for i in a]
    a = sample['pred'].tolist()
    hypo = [i.split() for i in a]
    c_bleu = corpus_bleu(ref, hypo)
    l.append(c_bleu)
    return l

X, y = [], []

in_df, out_df = df[df['label'] == 0], df[df['label'] == 1]
for i in range(3000):
    in_sample, out_sample = in_df.sample(500), out_df.sample(500)
    in_features, out_features = group_features(in_sample), group_features(out_sample)

    X.append(in_features)
    X.append(out_features)
    y.append(0)
    y.append(1)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=902022)

In [28]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print('The accuracy of Random Forest Classifier on training set:', rfc.score(X_train, y_train))
print('The accuracy of Random Forest Classifier on testing set:', rfc.score(X_test, y_test))

The accuracy of Random Forest Classifier on training set: 1.0
The accuracy of Random Forest Classifier on testing set: 0.8341666666666666


In [29]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
print('The accuracy of eXtreme Gradient Boosting Classifier on training set:', xgbc.score(X_train, y_train))
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))

/tmp2/b07902022/anaconda3/envs/SPML/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:56:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The accuracy of eXtreme Gradient Boosting Classifier on training set: 1.0
The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.8391666666666666


In [30]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(X_train, y_train)
print('The accuracy of MLP Classifier on training set:', clf.score(X_train, y_train))
print('The accuracy of MLP Classifier on testing set:', clf.score(X_test, y_test))


The accuracy of MLP Classifier on training set: 0.9860416666666667
The accuracy of MLP Classifier on testing set: 0.7375


/tmp2/b07902022/anaconda3/envs/SPML/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [31]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)
print('The accuracy of KNN on testing set:', classifier.score(X_train, y_train))
print('The accuracy of KNN on testing set:', classifier.score(X_test, y_test))

The accuracy of KNN on testing set: 0.7877083333333333
The accuracy of KNN on testing set: 0.6425


In [32]:
a_in_data = json.load(open('../Bob_translate_result/A_in_result.json'))
a_out_data = json.load(open('../Bob_translate_result/A_out_result.json'))
a_data = list(a_in_data.values()) + list(a_out_data.values())
a_label = [0 for _ in range(len(list(a_in_data.values())))] + [1 for _ in range(len(list(a_out_data.values())))]
a_data_dict = {'ori': [a_data[i][0] for i in range(len(a_data))], 'gt': [a_data[i][1] for i in range(len(a_data))], 'pred': [a_data[i][2] for i in range(len(a_data))], 'label': a_label}
a_df = pd.DataFrame(a_data_dict)
a_df.head()

a_df['1-gram'] = a_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 1), axis=1)
a_df['2-gram'] = a_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 2), axis=1)
a_df['3-gram'] = a_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 3), axis=1)
a_df['4-gram'] = a_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 4), axis=1)
a_df['bleu'] = a_df.apply(lambda r: bleu_score(r['gt'], r['pred']), axis=1)
a_df.head()

,ori,gt,pred,label,1-gram,2-gram,3-gram,4-gram,bleu
0,第918(1994)號決議,RESOLUTION 918 (1994),RESOLUTION 918 (1994) CONCERNING THE QUESTION ...,0,0.187500,0.133333,0.071429,0.000000,0.060879
1,1994年5月17日安全理事會第3377次會議通過,Adopted by the Security Council at its 3377th ...,Adopted on 17 May 1994 by the Security Council...,0,0.764706,0.687500,0.600000,0.500000,0.630191
2,安全理事會，,"The Security Council,","The Security Council,See Official Records of t...",0,0.117647,0.062500,0.000000,0.000000,0.024325
3,重申其以往關於盧旺達局勢的所有決議，特別是成立聯合國盧旺達援助團(聯盧援助團)的1993年1...,Reaffirming all its previous resolutions on th...,Reaffirming all its previous resolutions on th...,0,0.890909,0.722222,0.603774,0.500000,0.533743
4,回顧安理會主席以安理會名義在1994年4月7日發表的聲明(S/PRST/ 1994/16)和...,Recalling the statements made by the President...,Recalling the statements made by the President...,0,0.720000,0.708333,0.695652,0.681818,0.701306


In [33]:
a_X, a_y = [], []

a_in_df, a_out_df = a_df[a_df['label'] == 0], a_df[a_df['label'] == 1]
for i in range(250):
    in_sample, out_sample = a_in_df.sample(500), a_out_df.sample(500)
    in_features, out_features = group_features(in_sample), group_features(out_sample)

    a_X.append(in_features)
    a_X.append(out_features)
    a_y.append(0)
    a_y.append(1)
print('The accuracy of Random Forest Classifier on Alice set:', rfc.score(a_X, a_y))
print('The accuracy of eXtreme Gradient Boosting Classifier on Alice set:', xgbc.score(a_X, a_y))
print('The accuracy of MLP Classifier on Alice set:', clf.score(a_X, a_y))
print('The accuracy of KNN on Alice set:', classifier.score(a_X, a_y))

The accuracy of Random Forest Classifier on Alice set: 0.5
The accuracy of eXtreme Gradient Boosting Classifier on Alice set: 0.5
The accuracy of MLP Classifier on Alice set: 0.488
The accuracy of KNN on Alice set: 0.506


In [34]:
ood_data = json.load(open('../Bob_translate_result/A_ood_result.json'))
ood_data = list(ood_data.values())
ood_label = [1 for _ in range(len(ood_data))]
ood_data_dict = {'ori': [ood_data[i][0] for i in range(len(ood_data))], 'gt': [ood_data[i][1] for i in range(len(ood_data))], 'pred': [ood_data[i][2] for i in range(len(ood_data))], 'label': ood_label}
ood_df = pd.DataFrame(ood_data_dict)
ood_df.head()

ood_df['1-gram'] = ood_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 1), axis=1)
ood_df['2-gram'] = ood_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 2), axis=1)
ood_df['3-gram'] = ood_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 3), axis=1)
ood_df['4-gram'] = ood_df.apply(lambda r: modified_n_gram_precisions(r['gt'], r['pred'], 4), axis=1)
ood_df['bleu'] = ood_df.apply(lambda r: bleu_score(r['gt'], r['pred']), axis=1)
ood_df.head()

,ori,gt,pred,label,1-gram,2-gram,3-gram,4-gram,bleu
0,Brion VIBBER,Brion VIBBER,Breon VIBBER Qualified by the Secretary-Genera...,1,0.100000,0.0,0.0,0.0,0.021105
1,Mountain,Mountain,"Mount'ain, please. No, no, you don't. You know...",1,0.000000,0.0,0.0,0.0,0.000000
2,數學,Mathematics,Mathematics (mathematicals).. 6 - 4 4 - 5 4,1,0.111111,0.0,0.0,0.0,0.023980
3,目錄,Contents,Contents. 7 - 7 2 - 5 3 - 4 4 - 3 3 — 5 4,1,0.000000,0.0,0.0,0.0,0.000000
4,哲學,Philosophy,Philosophy (Psythropology).. 6 - 4 4 3,1,0.142857,0.0,0.0,0.0,0.033032


In [35]:
ood_X, ood_y = [], []

for i in range(250):
    sample = ood_df.sample(500)
    features = group_features(sample)

    ood_X.append(features)
    ood_y.append(1)
print('The accuracy of Random Forest Classifier on Alice set:', rfc.score(ood_X, ood_y))
print('The accuracy of eXtreme Gradient Boosting Classifier on Alice set:', xgbc.score(ood_X, ood_y))
print('The accuracy of MLP Classifier on Alice set:', clf.score(ood_X, ood_y))
print('The accuracy of KNN on Alice set:', classifier.score(ood_X, ood_y))

The accuracy of Random Forest Classifier on Alice set: 0.54
The accuracy of eXtreme Gradient Boosting Classifier on Alice set: 1.0
The accuracy of MLP Classifier on Alice set: 1.0
The accuracy of KNN on Alice set: 1.0


In [36]:
X, y = [], []

for i in range(6000):
    sample = df.sample(500)
    features = group_features(sample)
    count = sample[sample['label'] == 1].shape[0]

    X.append(features)
    y.append(count)

In [37]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=902022)
rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
print('The accuracy of Random Forest Regressor on training set:', rfc.score(X_train, y_train))
print('The accuracy of Random Forest Regressor on testing set:', rfc.score(X_test, y_test))
xgbc = XGBRegressor()
xgbc.fit(X_train, y_train)
print('The accuracy of eXtreme Gradient Boosting Regressor on training set:', xgbc.score(X_train, y_train))
print('The accuracy of eXtreme Gradient Boosting Regressor on testing set:', xgbc.score(X_test, y_test))
clf = MLPRegressor()
clf.fit(X_train, y_train)
print('The accuracy of MLP Regressor on training set:', clf.score(X_train, y_train))
print('The accuracy of MLP Regressor on testing set:', clf.score(X_test, y_test))
Regressor = KNeighborsRegressor(n_neighbors=5)
Regressor.fit(X_train, y_train)
print('The accuracy of KNN on testing set:', Regressor.score(X_train, y_train))
print('The accuracy of KNN on testing set:', Regressor.score(X_test, y_test))

The accuracy of Random Forest Regressor on training set: 0.8594344630948925
The accuracy of Random Forest Regressor on testing set: -0.008732927806421653
The accuracy of eXtreme Gradient Boosting Regressor on training set: 0.8635444055243859
The accuracy of eXtreme Gradient Boosting Regressor on testing set: -0.17696602660158112
The accuracy of MLP Regressor on training set: 0.02496164995828387
The accuracy of MLP Regressor on testing set: -0.011530413005686801
The accuracy of KNN on testing set: 0.192633972041637
The accuracy of KNN on testing set: -0.1922302137942502


In [38]:
a_X, a_y = [], []

for i in range(500):
    sample = a_df.sample(500)
    features = group_features(sample)
    count = sample[sample['label'] == 1].shape[0]

    a_X.append(features)
    a_y.append(count)
print('The accuracy of Random Forest Regressor on Alice set:', rfc.score(a_X, a_y))
print('The accuracy of eXtreme Gradient Boosting Regressor on Alice set:', xgbc.score(a_X, a_y))
print('The accuracy of MLP Regressor on Alice set:', clf.score(a_X, a_y))
print('The accuracy of KNN on Alice set:', Regressor.score(a_X, a_y))

The accuracy of Random Forest Regressor on Alice set: -0.02238206949363919
The accuracy of eXtreme Gradient Boosting Regressor on Alice set: -0.2007272231121653
The accuracy of MLP Regressor on Alice set: -0.012497173484122381
The accuracy of KNN on Alice set: -0.16813393673784005


In [40]:
ood_X, ood_y = [], []

for i in range(250):
    sample = ood_df.sample(500)
    features = group_features(sample)

    ood_X.append(features)
    ood_y.append(500)
print('The accuracy of Random Forest Regressor on Alice set:', rfc.score(ood_X, ood_y))
print('The accuracy of eXtreme Gradient Boosting Regressor on Alice set:', xgbc.score(ood_X, ood_y))
print('The accuracy of MLP Regressor on Alice set:', clf.score(ood_X, ood_y))
print('The accuracy of KNN on Alice set:', Regressor.score(ood_X, ood_y))

The accuracy of Random Forest Regressor on Alice set: 0.0
The accuracy of eXtreme Gradient Boosting Regressor on Alice set: 0.0
The accuracy of MLP Regressor on Alice set: 0.0
The accuracy of KNN on Alice set: 0.0
